## Load data

In [1]:
import numpy as np
import scipy.io.arff as sparff
import random
import math 

nEpochs = 200
nfolds = 30
NNOutput = {}

def loadData(filename):
    # reading the training data
    data, attribute = sparff.loadarff(filename)
    
    #getting size of attributes and instances
    numFeatures = len(attribute.names())-1
    numInstances = len(data)
    classLabel = attribute[attribute.names()[-1]]
    
    X, Y = [], []
    for i in range(numInstances):
        #print data[i][0]
        # convert labels to 0-1 encoding
        Y.append(classLabel[1].index(data[i][-1]))
        
        # create feature vector representation for each isntance
        featureVector = []
        for j in range(numFeatures):
            featureVector.append(data[i][j])
        X.append(featureVector)

    # get the number of possible values of each feature
#     numvals = []
#     for n in range(len(attribute.names())):
#         numvals.append(len(attribute[attribute.names()[n]][1]))
    npX = np.array(X)
    npY = np.array(Y)
    return npX, npY, attribute, numFeatures, numInstances, classLabel

In [2]:
X, Y, metadata, numFeatures, numInstances, classLabel = loadData('sonar.arff')

### Initialize weights and a helper function for sigmoid function

In [3]:
def sigmoidFunction(input):
    return np.divide(1.0,(np.add(1.0,np.exp(-input))))

def initializaWeights(numFeatures, numHidden):
    #print numHidden
    weightsHidden = []
    weightsOutput = []
    weightsBias = []
    weightsHidden.append(np.random.uniform(-0.1, 0.1, (numFeatures, numHidden)))
    weightsOutput.append(np.random.uniform(-0.1, 0.1, numHidden))
    
    oBias = random.uniform(-0.1,0.1)
    hBias = random.uniform(-0.1,0.1)
    weightsBias.append(oBias)
    weightsBias.append(hBias)
    
    #print np.shape(weights[0])
    return weightsHidden, weightsOutput, weightsBias

In [4]:
weightsHidden, weightsOutput, weightsBias = initializaWeights(numFeatures, numFeatures)
# weightsHidden= np.array([[.15,.2],[.25,.3]]) 
# weightsOutput = np.array([[.4,.45]]) 
len(weightsHidden[0]), len(weightsOutput[0]), numFeatures, len(weightsBias)

(60, 60, 60, 2)

### train model

In [5]:
def trainModel(X, Y, weightsHidden, weightsOutput, weightsBias, foldInstanceIndices):
    
    randomIndices = np.random.permutation(foldInstanceIndices)
    
#     X_train = X[list(randomIndices),:]
#     Y_train = Y[list(randomIndices)]
    for i in randomIndices:
        
        ## forward pass
        inputRow = np.array(X[i])
        
        ##hidden layer output calculation
        hiddenLayer = []
        for j in range(0,len(weightsHidden)):
            sumWeightsHidden = np.dot(inputRow,weightsHidden[j]) + weightsBias[1]
            hiddenOut = sigmoidFunction(sumWeightsHidden)
            #print sumWeightsHidden, hiddenOut
            hiddenLayer.append(hiddenOut)
        #print '\n'

        ##output layer output calculation
        sumWeightsOutput = np.dot(np.array(hiddenLayer),weightsOutput[0]) + weightsBias[0]
        outputOut = sigmoidFunction(sumWeightsOutput)
        #print sumWeightsOutput,outputOut

        
        error = 0.5 * math.pow((Y[i]-outputOut),2)
        weightsHidden, weightsOutput, weightsBias = backProp(inputRow, Y[i], hiddenLayer, outputOut, weightsOutput, weightsHidden, weightsBias)
        
    return weightsHidden, weightsOutput, weightsBias, error

In [6]:
def backProp(inputRow, y, hiddenLayer, outputOut, weightsOutput, weightsHidden, weightsBias):
    ## backpropagation
    lrate = 0.1

    #Output layer weight update
    delta_o = outputOut*(1-outputOut)*(y-outputOut)
    #print len(delta_o)
    wtsOut_gradient = np.multiply((lrate*delta_o),hiddenLayer)
    weightsOutput = np.add (weightsOutput, wtsOut_gradient)

    #Hidden layer weight update
    for j in range(0,len(weightsHidden)):
        delta_h = hiddenLayer[j]* (1-hiddenLayer[j])*delta_o*weightsOutput[0][j]
        #print len(delta_h)
        wtsHid_gradient = np.multiply((lrate*delta_h),inputRow)
        weightsHidden [j] =np.add (weightsHidden[j], wtsHid_gradient)
        
    #Bias weights update
    weightsBias[0] = lrate*delta_o
    weightsBias[1] = lrate*delta_h
    
    return weightsHidden, weightsOutput, weightsBias


# for e in range(nEpochs):
#     weightsHidden, weightsOutput, error = trainModel(X,Y,weightsHidden,weightsOutput)
#     print e, error
#     #print weightsHid, weightsOut

### test model

In [7]:
def predictClass(inputRow,weightsHidden,weightsOutput, weightsBias):
    hiddenLayer = []
    for j in range(0,len(weightsHidden)):
        sumWeightsHidden = np.dot(inputRow,weightsHidden[j]) + weightsBias[1]
        hiddenOut = sigmoidFunction(sumWeightsHidden)
        hiddenLayer.append(hiddenOut)

    ##output layer output calculation
    sumWeightsOutput = np.dot(np.array(hiddenLayer),weightsOutput[0]) + weightsBias[0]
    outputOut = sigmoidFunction(sumWeightsOutput)
    return outputOut

def testModel(X,Y,weightsHidden,weightsOutput, weightsBias, foldInstanceIndices,fold):
    
    for idx in list(foldInstanceIndices):
        
        row = np.array(X[idx])
        output = predictClass(row,weightsHidden,weightsOutput,weightsBias)
        #print 'prediction:', output[0]
        if output > 0.5:
            predictedClass = classLabel[1][1]
        else: 
            predictedClass = classLabel[1][0]
        
        if Y[idx]==1:
            actualClass = classLabel[1][1]
        else: 
            actualClass = classLabel[1][0]
            
        if NNOutput.has_key(idx):
            print 'error'
        else:
            NNOutput[idx] = [fold+1, predictedClass, actualClass, output[0]]
        

### Stratifying dataset

In [30]:
def stratifiedSample(X, Y, nfolds):
    numNegInstance = list(Y).count(0)
    numPosInstance = list(Y).count(1)
    numNegInstance, numPosInstance
    negIndices = np.where(Y==0)[0]
    posIndices = np.where(Y==1)[0]

    posInstanceInFold = (numPosInstance/nfolds)
    negInstanceInFold = (numNegInstance/nfolds)

    foldInstanceIndices = []
    i= j = 0
    for f in range(0,nfolds):
    #     print posIndices[i:(i+posInstanceInFold)]
    #     print negIndices[j:(j+negInstanceInFold)]
        foldInstanceIndices.append(np.append(posIndices[i:(i+posInstanceInFold)],negIndices[j:(j+negInstanceInFold)]))
        #foldLabelIndices.append(np.append(posIndices[i:(i+posInstanceInFold)],negIndices[j:(j+negInstanceInFold)]))
        i+=posInstanceInFold
        j+=negInstanceInFold
    #foldInstanceIndices
    
    return foldInstanceIndices, numPosInstance, numNegInstance

In [9]:
def crossValidation(X, Y, foldInstanceIndices, nfolds, nEpochs, weightsHidden, weightsOutput, weightsBias):
    for f in range(0,nfolds):
        testFold = f
        for t in range(0,len(foldInstanceIndices)):
            if t==testFold:
                continue

            for e in range(nEpochs):
                weightsHidden, weightsOutput, weightsBias, error = trainModel(X,Y,weightsHidden,weightsOutput, weightsBias, foldInstanceIndices[t])
                #print e, error

        testModel(X,Y,weightsHidden,weightsOutput, weightsBias, foldInstanceIndices[testFold],testFold)


In [10]:
def printOutput(numInstances):

    count =0
    tp, tn, fp, fn = 0, 0, 0, 0
    for i in range(0,numInstances):
        if NNOutput.has_key(i):
            count+=1
            values = NNOutput.get(i)
            print "{} {} {} {}".format(*values)

            if values[1]=='Rock':
                if values[2] == 'Rock':
                    tn += 1
                else:
                    fp += 1
            else:
                if values[1] == 'Mine':
                    tp += 1
                else:
                    fn += 1
    accuracy = (float)(tp + tn) / (float)(tp + tn + fp + fn) 
    print accuracy
            

In [31]:
foldInstanceIndices, p, n = stratifiedSample(X, Y, 10)
# crossValidation(X, Y, foldInstanceIndices, nfolds, nEpochs, weightsHidden, weightsOutput, weightsBias)
# accuracy = printOutput(numInstances)

In [38]:
!python main.py neuralnet sonar.arff 10 0.1 100 

  File "neuralnet", line 1
    python main.py $1 $2 $3 $4
              ^
SyntaxError: invalid syntax


In [16]:
#ROCcurve()
[0.8846153846153846,
 0.9326923076923077,
 0.8798076923076923,
 0.9471153846153846]

208

In [32]:
p,n

(111, 97)